In [1]:
import os
import soundfile as sf
import numpy as np
from scipy.signal import butter, sosfilt

def bandpass_filter(data, samplerate, lowcut, highcut, order=5):
    sos = butter(order, [lowcut, highcut], btype='band', fs=samplerate, output='sos')
    return sosfilt(sos, data)

def filter_ogg(input_folder, lowcut=1000.0, highcut=8000.0, order=5):
    base_folder = os.path.dirname(input_folder)
    folder_name = os.path.basename(input_folder.rstrip("/\\"))
    output_folder = os.path.join(base_folder, folder_name + "_filtered")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".ogg"):
            in_path = os.path.join(input_folder, filename)
            out_path = os.path.join(output_folder, filename)

            try:
                data, sr = sf.read(in_path)
                if len(data.shape) == 2:
                    data = data.mean(axis=1)
                filtered_data = bandpass_filter(data, sr, lowcut, highcut, order)
                sf.write(out_path, filtered_data, sr)
                print(f"✔️ 필터링 완료: {filename}")
            except Exception as e:
                print(f"❌ 오류 발생 ({filename}): {e}")

    print(f"\n📁 필터링된 파일 저장 경로: {output_folder}")
    return output_folder


In [3]:
input_folder = "birdclef-2025//train_audio//21038"
filter_ogg(input_folder, lowcut=2000.0, highcut=4000.0, order=5)

✔️ 필터링 완료: iNat297879.ogg


: 